## Tugas 2

Rafly Faldiansyah Putra

210411100063

Preprocessing hasil crawling data dari jatim.tribunnews.com

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

# Fungsi untuk mendapatkan konten dari URL
def get_soup(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return BeautifulSoup(response.content, 'html.parser')
    except requests.RequestException as e:
        print(f"Kesalahan saat mengambil {url}: {e}")
        return None

# Fungsi untuk mendapatkan detail artikel dari halaman detail
def get_article_details(detail_url):
    detail_soup = get_soup(detail_url)
    if detail_soup:
        # Ambil isi berita
        content = ' '.join([p.text for p in detail_soup.find_all('p')])

        # Ambil tanggal publikasi
        date_tag = detail_soup.find('time')
        date = date_tag.text.strip() if date_tag else 'Tidak ada tanggal'

        # Ambil judul berita
        title_tag = detail_soup.find('h1')
        title = title_tag.text.strip() if title_tag else 'Tidak ada judul'

        # Ambil kategori dari breadcrumb
        breadcrumb = detail_soup.find('ul', {'class': 'breadcrumb'})
        category = breadcrumb.find_all('li')[-1].find('span').text.strip() if breadcrumb else 'Tidak ada kategori'

        return {
            'judul': title,
            'isi_berita': content,
            'tanggal': date,
            'kategori': category,
            'url': detail_url
        }
    return None

# Fungsi untuk mendapatkan artikel dari suatu kategori
def get_articles(category_url, category_name, max_articles=50):
    articles = []
    page = 1
    
    while len(articles) < max_articles:
        url = f'{category_url}?page={page}'  # Periksa pola yang benar untuk paginasi
        print(f"Mengambil: {url}")  # Output debug
        soup = get_soup(url)
        
        if soup is None:
            break
        
        # Cari artikel di halaman
        article_list = soup.find_all('h3')  # Berdasarkan tata letak yang diamati
        
        if not article_list:
            print(f"Tidak ada artikel ditemukan di halaman {page}.")
            break
        
        for article in article_list:
            if len(articles) >= max_articles:
                break
            
            # Ambil URL detail artikel
            title_tag = article.find('a')
            detail_url = title_tag['href'] if title_tag else None
            
            if detail_url:
                if not detail_url.startswith('http'):
                    detail_url = f'https://jatim.tribunnews.com{detail_url}'
                # Buka halaman detail artikel
                article_details = get_article_details(detail_url)
                if article_details:
                    # Tambahkan ke daftar artikel
                    articles.append(article_details)
        
        page += 1
        time.sleep(2)  # Beri jeda agar tidak terlalu cepat melakukan permintaan
        
    return articles

# URL Kategori
categories = {
    'Travel': 'https://jatim.tribunnews.com/travel',
    'Sport': 'https://jatim.tribunnews.com/sport'
}

# Mengumpulkan semua data
all_articles = []
for category_name, category_url in categories.items():
    print(f"Menambang kategori {category_name}...")
    articles = get_articles(category_url, category_name, max_articles=50)
    all_articles.extend(articles)

# Simpan ke dalam DataFrame
df = pd.DataFrame(all_articles)

# Simpan ke dalam file CSV
df.to_csv('tribunnews_articles.csv', index=False)

# Tampilkan 10 data pertama dalam bentuk tabel
print(df.head(10))

print("Proses penambangan data selesai, data tersimpan dalam 'tribunnews_articles.csv' dan 10 data pertama ditampilkan.")


Menambang kategori Travel...
Mengambil: https://jatim.tribunnews.com/travel?page=1
Mengambil: https://jatim.tribunnews.com/travel?page=2
Menambang kategori Sport...
Mengambil: https://jatim.tribunnews.com/sport?page=1
Mengambil: https://jatim.tribunnews.com/sport?page=2
                                               judul  \
0  Cicipi Putu Mayang khas Betawi, Kue Lembut Ber...   
1  Kolam Renang Jambore Playland Surabaya Diresmi...   
2  The Alana Surabaya Beri Pengalaman Makan Siang...   
3  Rayakan Anniversary ke-45, Mondial Gandeng Nic...   
4  Volume Air Telaga Sarangan Magetan yang Menyus...   
5  Kampung Kayutangan Heritage Kota Malang Diserb...   
6  Selama Libur Panjang Akhir Pekan, Penumpang KA...   
7  Agoda Ungkap 5 Destinasi Favorit saat Libur Ma...   
8  Libur Panjang Maulid Nabi, PT KAI Daop 8 Surab...   
9  Mie Celor khas Palembang Kini Bisa Dinikmati d...   

                                          isi_berita  \
0  Laporan Wartawan TribunJatim.com, Nurika Anisa...   


In [2]:
df=pd.read_csv("tribunnews_articles.csv")
df.head(1000)

,judul,isi_berita,tanggal,kategori,url
0,"Cicipi Putu Mayang khas Betawi, Kue Lembut Ber...","Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Sabtu, 21 September 2024 08:36 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/21/cicipi...
1,Kolam Renang Jambore Playland Surabaya Diresmi...,"Laporan Wartawan Tribun Jatim Network, Bobby C...","Tayang: Sabtu, 21 September 2024 08:08 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/21/kolam-...
2,The Alana Surabaya Beri Pengalaman Makan Siang...,"Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Rabu, 18 September 2024 09:03 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/18/the-al...
3,"Rayakan Anniversary ke-45, Mondial Gandeng Nic...","Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Selasa, 17 September 2024 12:01 WIB",Shopping,https://jatim.tribunnews.com/2024/09/17/rayaka...
4,Volume Air Telaga Sarangan Magetan yang Menyus...,"Laporan Wartawan Tribun Jatim Network, Febrian...","Tayang: Selasa, 17 September 2024 09:21 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/17/volume...
...,...,...,...,...,...
95,Tim Voli Indoor Putri Jawa Timur Juarai PON XX...,"Laporan Wartawan Tribun Jatim Network, Khairul...","Tayang: Jumat, 20 September 2024 08:05 WIB",Olahraga Lain,https://jatim.tribunnews.com/2024/09/20/tim-vo...
96,"Kemenangan Srikandi Smada, Didorong oleh Kreat...","TRIBUNJATIM.COM, SURABAYA - Pertandingan tim b...","Tayang: Kamis, 19 September 2024 18:37 WIB",Basket,https://jatim.tribunnews.com/2024/09/19/kemena...
97,"Rela Resign Kerja Demi Latihan Keras, Atlet Pe...","Laporan Wartawan Tribun Jatim Network, Anggit ...","Tayang: Kamis, 19 September 2024 14:24 WIB",Olahraga Lain,https://jatim.tribunnews.com/2024/09/19/rela-r...
98,Coach Aries Ungkap Kemarahan ke Skuad NSA Usai...,"TRIBUNJATIM.COM, SURABAYA - Setelah kemenangan...","Tayang: Rabu, 18 September 2024 21:02 WIB",Basket,https://jatim.tribunnews.com/2024/09/18/coach-...


In [3]:
import pandas as pd

df = pd.read_csv("tribunnews_articles.csv")

df.head()

,judul,isi_berita,tanggal,kategori,url
0,"Cicipi Putu Mayang khas Betawi, Kue Lembut Ber...","Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Sabtu, 21 September 2024 08:36 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/21/cicipi...
1,Kolam Renang Jambore Playland Surabaya Diresmi...,"Laporan Wartawan Tribun Jatim Network, Bobby C...","Tayang: Sabtu, 21 September 2024 08:08 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/21/kolam-...
2,The Alana Surabaya Beri Pengalaman Makan Siang...,"Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Rabu, 18 September 2024 09:03 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/18/the-al...
3,"Rayakan Anniversary ke-45, Mondial Gandeng Nic...","Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Selasa, 17 September 2024 12:01 WIB",Shopping,https://jatim.tribunnews.com/2024/09/17/rayaka...
4,Volume Air Telaga Sarangan Magetan yang Menyus...,"Laporan Wartawan Tribun Jatim Network, Febrian...","Tayang: Selasa, 17 September 2024 09:21 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/17/volume...


## **Preprocessing**

Preprocessing adalah proses membersihkan dan mempersiapkan data mentah agar siap digunakan oleh model machine learning. Ini meliputi penanganan data yang hilang, normalisasi, mengubah data kategori menjadi angka, dan membersihkan teks. Tujuannya agar data lebih mudah dipahami dan diolah oleh model untuk hasil yang lebih akurat, Berikut adalah beberapa langkah umum dalam pre-processing teks:

### Cleansing

Proses cleansing data adalah tahap pembersihan teks dari elemen-elemen yang tidak relevan terhadap hasil klasifikasi sentimen. Beberapa komponen yang tidak berpengaruh terhadap sentimen, seperti URL, tag HTML, emoji, simbol, angka, dan tanda baca (~!@#$%^&*{}<>:|), dihapus dari dokumen ulasan. Elemen-elemen tersebut dihilangkan untuk mengurangi kebisingan (noise) dalam data. Setelah dihapus, elemen ini digantikan dengan spasi agar struktur kalimat tetap terjaga. Dengan demikian, data menjadi lebih fokus pada kata-kata yang relevan untuk menentukan sentimen, sehingga membantu meningkatkan akurasi model prediksi sentimen.

In [4]:
import re
import pandas as pd
import nltk
import string

def remove_url(text):
    #Fungsi untuk menghapus URL dari teks.
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

def remove_html(text):
    #Fungsi untuk menghapus tag HTML dari teks.
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)

def remove_emoji(text):

    #Fungsi untuk menghapus emoji dari teks.
    emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emotikon wajah
        u"\U0001F300-\U0001F5FF"  # simbol & gambar
        u"\U0001F680-\U0001F6FF"  # transportasi & simbol
        u"\U0001F1E0-\U0001F1FF"  # bendera negara
        "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

def remove_numbers(text):
    #Fungsi untuk menghapus angka dari teks.
    return re.sub(r'\d+', '', text)

def remove_symbols(text):
    #Fungsi untuk menghapus simbol dan karakter khusus dari teks.
    return re.sub(r'[^a-zA-Z0-9\s]', '', text)

# Asumsikan df adalah DataFrame yang berisi data CNN (judul, berita, tanggal, kategori)
# Contoh: df = pd.read_csv('berita-cnn.csv')

# Terapkan fungsi cleansing untuk kolom 'berita'
df['berita_clean'] = df['isi_berita'].apply(remove_url)
df['berita_clean'] = df['berita_clean'].apply(remove_html)
df['berita_clean'] = df['berita_clean'].apply(remove_emoji)
df['berita_clean'] = df['berita_clean'].apply(remove_symbols)
df['berita_clean'] = df['berita_clean'].apply(remove_numbers)



# Tampilkan beberapa baris dari hasil yang sudah dibersihkan
df.head(5)


,judul,isi_berita,tanggal,kategori,url,berita_clean
0,"Cicipi Putu Mayang khas Betawi, Kue Lembut Ber...","Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Sabtu, 21 September 2024 08:36 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/21/cicipi...,Laporan Wartawan TribunJatimcom Nurika Anisa T...
1,Kolam Renang Jambore Playland Surabaya Diresmi...,"Laporan Wartawan Tribun Jatim Network, Bobby C...","Tayang: Sabtu, 21 September 2024 08:08 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/21/kolam-...,Laporan Wartawan Tribun Jatim Network Bobby Co...
2,The Alana Surabaya Beri Pengalaman Makan Siang...,"Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Rabu, 18 September 2024 09:03 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/18/the-al...,Laporan Wartawan TribunJatimcom Nurika Anisa T...
3,"Rayakan Anniversary ke-45, Mondial Gandeng Nic...","Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Selasa, 17 September 2024 12:01 WIB",Shopping,https://jatim.tribunnews.com/2024/09/17/rayaka...,Laporan Wartawan TribunJatimcom Nurika Anisa T...
4,Volume Air Telaga Sarangan Magetan yang Menyus...,"Laporan Wartawan Tribun Jatim Network, Febrian...","Tayang: Selasa, 17 September 2024 09:21 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/17/volume...,Laporan Wartawan Tribun Jatim Network Febriant...


### CASE FOLDING

Pada tahap case folding, semua huruf kapital dalam dokumen ulasan diubah menjadi huruf kecil, atau disebut lowercase. Tujuan dari langkah ini adalah untuk menghilangkan redundansi data yang hanya disebabkan oleh perbedaan kapitalisasi. Misalnya, kata "Ekonomi" dan "ekonomi" secara teknis sama dalam analisis teks, namun tanpa case folding, komputer akan menganggapnya berbeda. Dengan mengonversi seluruh teks menjadi huruf kecil, semua variasi penulisan diseragamkan, sehingga mencegah duplikasi penghitungan atau kesalahan dalam interpretasi data.

In [5]:
def case_folding(text):
    if isinstance(text, str):
      lowercase_text = text.lower()
      return lowercase_text
    else :
      return text

df ['case_folding'] = df['berita_clean'].apply(case_folding)

df.head(5)

,judul,isi_berita,tanggal,kategori,url,berita_clean,case_folding
0,"Cicipi Putu Mayang khas Betawi, Kue Lembut Ber...","Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Sabtu, 21 September 2024 08:36 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/21/cicipi...,Laporan Wartawan TribunJatimcom Nurika Anisa T...,laporan wartawan tribunjatimcom nurika anisa t...
1,Kolam Renang Jambore Playland Surabaya Diresmi...,"Laporan Wartawan Tribun Jatim Network, Bobby C...","Tayang: Sabtu, 21 September 2024 08:08 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/21/kolam-...,Laporan Wartawan Tribun Jatim Network Bobby Co...,laporan wartawan tribun jatim network bobby co...
2,The Alana Surabaya Beri Pengalaman Makan Siang...,"Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Rabu, 18 September 2024 09:03 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/18/the-al...,Laporan Wartawan TribunJatimcom Nurika Anisa T...,laporan wartawan tribunjatimcom nurika anisa t...
3,"Rayakan Anniversary ke-45, Mondial Gandeng Nic...","Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Selasa, 17 September 2024 12:01 WIB",Shopping,https://jatim.tribunnews.com/2024/09/17/rayaka...,Laporan Wartawan TribunJatimcom Nurika Anisa T...,laporan wartawan tribunjatimcom nurika anisa t...
4,Volume Air Telaga Sarangan Magetan yang Menyus...,"Laporan Wartawan Tribun Jatim Network, Febrian...","Tayang: Selasa, 17 September 2024 09:21 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/17/volume...,Laporan Wartawan Tribun Jatim Network Febriant...,laporan wartawan tribun jatim network febriant...


### TOKENIZATION

Tokenization adalah tahap di mana setiap kata dalam sebuah dokumen dipecah menjadi unit-unit kata yang lebih kecil, atau disebut token. Proses ini memisahkan kata-kata berdasarkan spasi, sehingga setiap kata yang terpisah oleh spasi dianggap sebagai token tersendiri. Sebagai contoh, kalimat "Upaya agar ekonomi stabil" akan diuraikan menjadi token ["Upaya", "agar", "ekonomi", "stabil"].

In [6]:
def tokenize(text):
    tokens = text.split()
    return tokens

df['tokenize'] = df['case_folding'].apply(tokenize)

df.head(5)

,judul,isi_berita,tanggal,kategori,url,berita_clean,case_folding,tokenize
0,"Cicipi Putu Mayang khas Betawi, Kue Lembut Ber...","Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Sabtu, 21 September 2024 08:36 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/21/cicipi...,Laporan Wartawan TribunJatimcom Nurika Anisa T...,laporan wartawan tribunjatimcom nurika anisa t...,"[laporan, wartawan, tribunjatimcom, nurika, an..."
1,Kolam Renang Jambore Playland Surabaya Diresmi...,"Laporan Wartawan Tribun Jatim Network, Bobby C...","Tayang: Sabtu, 21 September 2024 08:08 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/21/kolam-...,Laporan Wartawan Tribun Jatim Network Bobby Co...,laporan wartawan tribun jatim network bobby co...,"[laporan, wartawan, tribun, jatim, network, bo..."
2,The Alana Surabaya Beri Pengalaman Makan Siang...,"Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Rabu, 18 September 2024 09:03 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/18/the-al...,Laporan Wartawan TribunJatimcom Nurika Anisa T...,laporan wartawan tribunjatimcom nurika anisa t...,"[laporan, wartawan, tribunjatimcom, nurika, an..."
3,"Rayakan Anniversary ke-45, Mondial Gandeng Nic...","Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Selasa, 17 September 2024 12:01 WIB",Shopping,https://jatim.tribunnews.com/2024/09/17/rayaka...,Laporan Wartawan TribunJatimcom Nurika Anisa T...,laporan wartawan tribunjatimcom nurika anisa t...,"[laporan, wartawan, tribunjatimcom, nurika, an..."
4,Volume Air Telaga Sarangan Magetan yang Menyus...,"Laporan Wartawan Tribun Jatim Network, Febrian...","Tayang: Selasa, 17 September 2024 09:21 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/17/volume...,Laporan Wartawan Tribun Jatim Network Febriant...,laporan wartawan tribun jatim network febriant...,"[laporan, wartawan, tribun, jatim, network, fe..."


### STOPWORD REMOVAL

Stopword removal adalah proses menghapus kata-kata yang dianggap tidak penting atau tidak memiliki makna signifikan dalam analisis teks, seperti "dan," "di," "yang," atau "itu." Kata-kata ini sering muncul dalam kalimat tetapi tidak memberikan informasi penting untuk pemrosesan atau analisis lebih lanjut. Dengan menghapus stopwords, data teks menjadi lebih ringkas dan fokus hanya pada kata-kata yang memiliki bobot lebih besar dalam analisis, seperti saat melakukan klasifikasi atau pemodelan teks.

In [7]:
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('indonesian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rafly\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
def remove_stopwords(text):
  return [word for word in text if word not in stop_words]

df['stopword_removal'] = df['tokenize'].apply(lambda x: ' '.join(remove_stopwords(x)))


df.to_csv("preprocessing-cnnnews.csv", encoding='utf8', index=False)
df.head(5)

,judul,isi_berita,tanggal,kategori,url,berita_clean,case_folding,tokenize,stopword_removal
0,"Cicipi Putu Mayang khas Betawi, Kue Lembut Ber...","Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Sabtu, 21 September 2024 08:36 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/21/cicipi...,Laporan Wartawan TribunJatimcom Nurika Anisa T...,laporan wartawan tribunjatimcom nurika anisa t...,"[laporan, wartawan, tribunjatimcom, nurika, an...",laporan wartawan tribunjatimcom nurika anisa t...
1,Kolam Renang Jambore Playland Surabaya Diresmi...,"Laporan Wartawan Tribun Jatim Network, Bobby C...","Tayang: Sabtu, 21 September 2024 08:08 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/21/kolam-...,Laporan Wartawan Tribun Jatim Network Bobby Co...,laporan wartawan tribun jatim network bobby co...,"[laporan, wartawan, tribun, jatim, network, bo...",laporan wartawan tribun jatim network bobby co...
2,The Alana Surabaya Beri Pengalaman Makan Siang...,"Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Rabu, 18 September 2024 09:03 WIB",Kuliner,https://jatim.tribunnews.com/2024/09/18/the-al...,Laporan Wartawan TribunJatimcom Nurika Anisa T...,laporan wartawan tribunjatimcom nurika anisa t...,"[laporan, wartawan, tribunjatimcom, nurika, an...",laporan wartawan tribunjatimcom nurika anisa t...
3,"Rayakan Anniversary ke-45, Mondial Gandeng Nic...","Laporan Wartawan TribunJatim.com, Nurika Anisa...","Tayang: Selasa, 17 September 2024 12:01 WIB",Shopping,https://jatim.tribunnews.com/2024/09/17/rayaka...,Laporan Wartawan TribunJatimcom Nurika Anisa T...,laporan wartawan tribunjatimcom nurika anisa t...,"[laporan, wartawan, tribunjatimcom, nurika, an...",laporan wartawan tribunjatimcom nurika anisa t...
4,Volume Air Telaga Sarangan Magetan yang Menyus...,"Laporan Wartawan Tribun Jatim Network, Febrian...","Tayang: Selasa, 17 September 2024 09:21 WIB",Destinasi,https://jatim.tribunnews.com/2024/09/17/volume...,Laporan Wartawan Tribun Jatim Network Febriant...,laporan wartawan tribun jatim network febriant...,"[laporan, wartawan, tribun, jatim, network, fe...",laporan wartawan tribun jatim network febriant...


## **TF-IDF (Term Frequency-Inverse Document Frequency)**

TF-IDF adalah metode statistik yang digunakan untuk mengevaluasi pentingnya suatu kata dalam sebuah dokumen relatif terhadap koleksi dokumen lainnya. TF-IDF sering digunakan dalam tugas seperti penggalian teks, penambangan informasi, dan pemodelan pembelajaran mesin berbasis teks.
Term Frequency (TF), yang menghitung seberapa sering sebuah kata muncul dalam dokumen, dan Inverse Document Frequency (IDF), yang menilai seberapa jarang kata tersebut muncul di seluruh dokumen dalam koleksi.

TF-IDF bekerja dengan memberikan bobot lebih tinggi pada kata-kata yang sering muncul dalam sebuah dokumen, tetapi jarang muncul di dokumen lain, sehingga membantu mengidentifikasi kata-kata yang paling relevan.


In [9]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

df = pd.read_csv("preprocessing-cnnnews.csv")

# Menginisialisasi TfidfVectorizer
vectorizer = TfidfVectorizer()

# Menghitung TF-IDF
tfidf_matrix = vectorizer.fit_transform(df['stopword_removal'])

In [10]:
# Mengubah hasilnya menjadi DataFrame
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=vectorizer.get_feature_names_out())
tfidf_df.head(10)

,abidin,academy,acara,access,aceh,acehsumut,act,adhy,aditiya,adonan,...,yogyakarta,yohanes,yola,yolandasari,yolk,you,youtube,yusman,zaqiya,zona
0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.200092,...,0.032210,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.057457
2,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0.0,0.0,0.041888,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
5,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.026773,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,0.0,0.0,0.057369,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.136410,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,0.0,0.0,0.000000,0.048523,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.026421,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.035386,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
